In [1]:
import pandas as pd

df = pd.read_csv("../data-output/preprocessed_2019_data.csv")

print(df)

      Match Date           Sport  Event Type  \
0     2019-03-31  doubles_squash  Club Match   
1     2019-03-29  doubles_squash  Club Match   
2     2019-03-28  doubles_squash      League   
3     2019-03-28  doubles_squash      League   
4     2019-03-28  doubles_squash      League   
...          ...             ...         ...   
1228  2018-04-04  doubles_squash  Tournament   
1229  2018-04-04  doubles_squash  Tournament   
1230  2018-04-04  doubles_squash  Tournament   
1231  2018-04-03  doubles_squash  Club Match   
1232  2018-04-01  doubles_squash  Club Match   

                             Event Name Match Status Game Scores  \
0               Vancouver Racquets Club       Normal    [(3, 0)]   
1               Vancouver Racquets Club       Normal    [(3, 0)]   
2       Vancouver Ladies Doubles League       Normal    [(1, 3)]   
3       Vancouver Ladies Doubles League       Normal    [(3, 1)]   
4       Vancouver Ladies Doubles League       Normal    [(3, 0)]   
...            

In [2]:
print(df.columns)

Index(['Match Date', 'Sport', 'Event Type', 'Event Name', 'Match Status',
       'Game Scores', 'Player A1', 'Player A1 Squash BC Number',
       'Player A1 Gender', 'Player A2', 'Player A2 Squash BC Number',
       'Player A2 Gender', 'Player B1', 'Player B1 Squash BC Number',
       'Player B1 Gender', 'Player B2', 'Player B2 Squash BC Number',
       'Player B2 Gender'],
      dtype='object')


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1233 entries, 0 to 1232
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Match Date                  1233 non-null   object 
 1   Sport                       1233 non-null   object 
 2   Event Type                  1233 non-null   object 
 3   Event Name                  1233 non-null   object 
 4   Match Status                1233 non-null   object 
 5   Game Scores                 1233 non-null   object 
 6   Player A1                   1233 non-null   object 
 7   Player A1 Squash BC Number  1229 non-null   float64
 8   Player A1 Gender            1233 non-null   object 
 9   Player A2                   1233 non-null   object 
 10  Player A2 Squash BC Number  1228 non-null   float64
 11  Player A2 Gender            1233 non-null   object 
 12  Player B1                   1233 non-null   object 
 13  Player B1 Squash BC Number  1219 

In [4]:
df = df[
    [
        "Match Date",
        "Game Scores",
        "Player A1",
        "Player A2",
        "Player B1",
        "Player B2",
    ]
]

In [5]:
import ast

df['Game Scores'] = df['Game Scores'].apply(ast.literal_eval) # To ensure that the arrays of tuples get read as arrays of tuples

In [6]:
# Initialize Elo ratings for each player
elo_ratings = {}

# Initialize parameters for the Elo system
starting_elo = 1200  # Starting Elo rating for new players
k_factor_base = 32  # Base K-factor for Elo rating adjustments


# Function to calculate K-factor based on Elo rating
def calculate_k_factor(elo_rating):
    if elo_rating < 2000:
        return 30
    elif elo_rating < 2400:
        return 20
    else:
        return 10


# Function to calculate expected win probability based on Elo ratings
def expected_win_probability(player_elo, opponent_elo):
    return 1 / (1 + 10 ** ((opponent_elo - player_elo) / 400))


# Iterate through each row (match) in the DataFrame
for index, row in df.iterrows():
    # Extract match details
    game_score = row["Game Scores"]
    
    # Track game scores and take highest difference
    a_wins = 0
    a_win_max = (0,0)
    b_wins = 0
    b_win_max = (0,0)

    for x in game_score:
        if x[0] > x[1]:
            a_wins += 1
            if (x[0] - x[1]) > (a_win_max[0] - a_win_max[1]):
                a_win_max = x
        elif x[0] < x[1]:
            b_wins += 1
            if (x[1] - x[0]) > (b_win_max[1] - b_win_max[0]):
                b_win_max = x

    if a_wins > b_wins:
        score = a_win_max
    else:
        score = b_win_max
        
    score_a = score[0]
    score_b = score[1]

    # Extract player names
    player_a1 = row["Player A1"]
    player_a2 = row["Player A2"]
    player_b1 = row["Player B1"]
    player_b2 = row["Player B2"]

    # Initialize Elo ratings for new players
    for player in [player_a1, player_a2, player_b1, player_b2]:
        if player not in elo_ratings:
            elo_ratings[player] = starting_elo

    # Calculate expected win probability for each team
    team_a_elo = (elo_ratings[player_a1] + elo_ratings[player_a2]) / 2
    team_b_elo = (elo_ratings[player_b1] + elo_ratings[player_b2]) / 2
    expected_win_a = expected_win_probability(team_a_elo, team_b_elo)
    expected_win_b = 1 - expected_win_a

    # Update Elo ratings based on actual outcome
    k_factor_a = calculate_k_factor(team_a_elo)
    k_factor_b = calculate_k_factor(team_b_elo)

    # Determine the score difference factor
    score_difference = abs(score_a - score_b)
    score_factor = 1 + (score_difference / 3)  # Adjust this factor as needed

    if score_a > score_b:
        # Team A won
        elo_ratings[player_a1] += k_factor_a * score_factor * (1 - expected_win_a)
        elo_ratings[player_a2] += k_factor_a * score_factor * (1 - expected_win_a)
        elo_ratings[player_b1] += k_factor_b * score_factor * (0 - expected_win_b)
        elo_ratings[player_b2] += k_factor_b * score_factor * (0 - expected_win_b)
    elif score_a < score_b:
        # Team B won
        elo_ratings[player_a1] += k_factor_a * score_factor * (0 - expected_win_a)
        elo_ratings[player_a2] += k_factor_a * score_factor * (0 - expected_win_a)
        elo_ratings[player_b1] += k_factor_b * score_factor * (1 - expected_win_b)
        elo_ratings[player_b2] += k_factor_b * score_factor * (1 - expected_win_b)
    else:
        # Draw
        pass

# Display or save the updated Elo ratings for each player
sorted_elo_ratings = sorted(elo_ratings.items(), key=lambda x: x[1], reverse=True)

for player, elo in sorted_elo_ratings:
    print(f"Player: {player}, Elo: {elo}")

Player: Andrew Smart, Elo: 1553.1373382140082
Player: Cathy Covernton, Elo: 1496.7970792943056
Player: Brian Covernton, Elo: 1478.5635215779432
Player: Brendan George, Elo: 1452.7825799516386
Player: Ed Berwick, Elo: 1432.068223357159
Player: Malcolm Moore, Elo: 1400.6405792272672
Player: Mark Heaney, Elo: 1395.0741741824688
Player: Carl McCreath, Elo: 1392.6091140413248
Player: Erik Nilsson, Elo: 1371.6778764813341
Player: David Goldsmith, Elo: 1371.6000655511634
Player: Derek Smith, Elo: 1370.6333768750922
Player: Claude Garceau, Elo: 1369.2337644703275
Player: Mike Lavigne, Elo: 1368.3710115563788
Player: Andrew Cunningham, Elo: 1363.9468209122263
Player: Chris Wilson, Elo: 1363.2729475184615
Player: Tessa Breukels, Elo: 1361.3873938790337
Player: Mikhail Din, Elo: 1361.3087733160035
Player: Jeff Boag, Elo: 1359.5044197510524
Player: Bob Hodgins, Elo: 1358.720117209354
Player: Robert Pacey, Elo: 1356.1932237498954
Player: Doug Buchanan, Elo: 1355.4309167077615
Player: Doug MacDougal

In [7]:
# Initialize win and loss counters
wins = 0
losses = 0
name = "Grant Bergman"

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    # Extract match details
    game_score = row["Game Scores"]
    # For multiple games, take the one with the largest score difference (??)
    score_diffs = [abs(x[0] - x[1]) for x in game_score]
    game = game_score[score_diffs.index(max(score_diffs))]
    score_a = game[0]
    score_b = game[1]

    # Extract player names (assuming the capitalization of 'p' in 'Player')
    player_a1 = row['Player A1']
    player_a2 = row['Player A2']
    player_b1 = row['Player B1']
    player_b2 = row['Player B2']

    # Check if player is part of the winning team
    if name in [player_a1, player_a2] and score_a > score_b:
        wins += 1
    elif name in [player_b1, player_b2] and score_b > score_a:
        wins += 1
    elif name in [player_a1, player_a2] or name in [player_b1, player_b2]:
        losses += 1

print(f"{name}'s Win-Loss Record: {wins}-{losses}")

Grant Bergman's Win-Loss Record: 3-1
